# tf.Variable

* 신경망에서 가중치와 같은 학습 가능한 parameter를 정의 할 때나 코드가 실행될 때 값이 변경 될 사항이 있을 때 유용함
  * cf) tf.constant: session을 통해 실행될 때 값이 변하지 않음 (immutable)
* `tf.Variable`은 사용하기 전에 꼭 **initializer**를 사용해야 함

In [ ]:
import tensorflow as tf

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

tf.set_random_seed(219)

In [ ]:
a = tf.Variable(2, name='scalar')
b = tf.Variable([2, 3], name='vector')
c = tf.Variable([[0, 1], [2, 3]], name='matrix')

In [ ]:
x = a + a
y = a + b
print(x)
print(y)

In [ ]:
with tf.Session(config=sess_config) as sess:
  print(sess.run(x))
  print(sess.run(y))
  # initialize를 하지 않아 에러가 난다

### Initialize

* 보통은 `tf.global_variables_initializer()`를 사용하여 모든 `tf.Variable`들은 한번에 초기화한다.
  * 각 변수를 인자로 넣어 각각 변수별로 initialize 할 수 있다.

#### 모든 변수 초기화

In [ ]:
init_op = tf.global_variables_initializer()
with tf.Session(config=sess_config) as sess:
  sess.run(init_op)
  print(sess.run(x))
  print(sess.run(y))

#### 변수 지정하여 초기화

* 변수 `a`와 `b`는 초기화 `c`는 초기화하지 않음
  * `a`, `b`: 정상
  * `c`: error

In [ ]:
# Initialize only a subset of variables
init_ab = tf.variables_initializer([a, b], name="init_ab")
with tf.Session(config=sess_config) as sess:
  sess.run(init_ab)
  print(sess.run(a))
  print(sess.run(b))
  print(sess.run(c)) # a, b는 initialize가 되어있고, c는 안되어있어서 에러가 난다

In [ ]:
# Initialize a single variable
W = tf.Variable(tf.zeros([3, 2]))
with tf.Session(config=sess_config) as sess:
  sess.run(W.initializer)
  print(sess.run(W))

### `tf.Variable.eval()`

* `with` 구문 안에서 `sess.run()` 대신에 `Tensor`에 직접 실행 명령을 할 수 있다.

In [ ]:
# Initialize a single variable
W = tf.Variable(tf.random_normal([3, 2]))
with tf.Session(config=sess_config) as sess:
  sess.run(W.initializer)
  print(W.eval())

### `tf.Variable.assign()`

In [ ]:
W = tf.Variable(10)
W.assign(100)
with tf.Session(config=sess_config) as sess:
  sess.run(W.initializer)
  print(W.eval())

In [ ]:
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session(config=sess_config) as sess:
  sess.run(W.initializer)
  sess.run(assign_op)
  print(W.eval())

In [ ]:
# create a variable whose original value is 2
my_var = tf.Variable(2, name="my_var")

# assign a * 2 to a and call that op a_times_two
my_var_times_two = my_var.assign(2 * my_var)

with tf.Session(config=sess_config) as sess:
  sess.run(my_var.initializer)
  print(sess.run(my_var_times_two)) # >> 4
  print(sess.run(my_var_times_two)) # >> 8
  print(sess.run(my_var_times_two)) # >> 16

## Two Sessions

* `tf.Session()`을 동시에 두개를 돌려보자
* 같은 변수 `W`가 서로 다른 Session에서 각각 다른 값을 가지고 있다

In [ ]:
W = tf.Variable(10)

sess1 = tf.Session(config=sess_config)
sess2 = tf.Session(config=sess_config)

sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10))) # >> 20
print(sess2.run(W.assign_sub(2))) # >> 8

print(sess1.run(W.assign_add(100))) # >> 120
print(sess2.run(W.assign_sub(50))) # >> -42

sess1.close()
sess2.close()